This script is used to calculate the emissions from fertilizer application and pesticides.

In [2]:
import pandas as pd
import bw2data as bd


In [3]:
# User input data from UI! MUST REPLACE WITH VARIABLE FROM KARINA
user_input_fert_type = "Urea"
user_input_fert_quantity_kg = 0.02

In [5]:
# Create dataframe for generic fertilizer content
data_fertilizer_content = {
    'fertilizer_type': ['Ammonium Sulphate', 'Ammonium chloride', 'Calcium Ammonium Nitrate', 
                        'Calcium Ammonium Nitrate', 'Calcium Nitrate', 'Urea', 
                        'Single superphosphate SSP 14%', 'Rock Phosphate (powder/granular)', 
                        'Potassium chloride (powder/granular)', 'Potassium Sulphate', 
                        'NPK 15-15-15', 'NPK 10-26-26'],
    'fert_N_content_%': [0.206, 0.250, 0.250, 0.260, 0.155, 0.460, 0.000, 0.000, 0.000, 0.000, 0.150, 0.100],
    'fert_P2O5_content_%': [0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.140, 0.180, 0.000, 0.000, 0.150, 0.221],
    'fert_K2O_content_%': [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.60, 0.50, 0.15, 0.26]
}

df_fertilizer_content = pd.DataFrame(data_fertilizer_content)

def calculate_individual_nutrient_contents(user_input_fert_type, user_input_fert_quantity_kg):
    fert_row = df_fertilizer_content[df_fertilizer_content['fertilizer_type'] == user_input_fert_type]
    
    if fert_row.empty:
        raise ValueError(f"Fertilizer type '{user_input_fert_type}' not found in the DataFrame.")
    
    nutrient_content_N = fert_row['fert_N_content_%'].values[0]
    nutrient_content_P = fert_row['fert_P2O5_content_%'].values[0]
    nutrient_content_K = fert_row['fert_K2O_content_%'].values[0]
      
    user_input_fert_kg_N = user_input_fert_quantity_kg * nutrient_content_N
    user_input_fert_kg_P = user_input_fert_quantity_kg * nutrient_content_P
    user_input_fert_kg_K = user_input_fert_quantity_kg * nutrient_content_K
    
    return user_input_fert_kg_N, user_input_fert_kg_P, user_input_fert_kg_K

# Function to calculate emissions
def calculate_emissions(user_input_fert_kg_N, user_input_fert_kg_P, EF_N2O_fert, EF_NH3_syn_fert, EF_NH3_org_fert, EF_NO3_fert, EF_NOx_fert, EF_P_fert):
    Emission_kg_N2O = user_input_fert_kg_N * EF_N2O_fert
    Emission_syn_kg_NH3 = user_input_fert_kg_N * EF_NH3_syn_fert
    Emission_org_kg_NH3 = user_input_fert_kg_N * EF_NH3_org_fert
    Emission_kg_NH3 = Emission_syn_kg_NH3 + Emission_org_kg_NH3
    Emission_kg_NO3 = user_input_fert_kg_N * EF_NO3_fert
    Emission_kg_NOx_N = (user_input_fert_kg_N - Emission_kg_NH3 * 17 / 14) * EF_NOx_fert
    Emission_kg_P = user_input_fert_kg_P * EF_P_fert
    
    return {
        'Emission_kg_N2O': Emission_kg_N2O,
        'Emission_kg_NH3': Emission_kg_NH3,
        'Emission_kg_NO3': Emission_kg_NO3,
        'Emission_kg_NOx_N': Emission_kg_NOx_N,
        'Emission_kg_P': Emission_kg_P
    }

# Nutrient contents calculation
user_input_fert_kg_N, user_input_fert_kg_P, user_input_fert_kg_K = calculate_individual_nutrient_contents(user_input_fert_type, user_input_fert_quantity_kg)

# Emission factors
EF_N2O_fert = 0.022
EF_NH3_syn_fert = 0.12
EF_NH3_org_fert = 0.24
EF_NO3_fert = 1.33
EF_NOx_fert = 0.012
EF_P_fert = 0.05

# Emissions calculation
emissions = calculate_emissions(user_input_fert_kg_N, user_input_fert_kg_P, EF_N2O_fert, EF_NH3_syn_fert, EF_NH3_org_fert, EF_NO3_fert, EF_NOx_fert, EF_P_fert)

# Display the results
for emission_type, value in emissions.items():
    print(f"{emission_type}: {value:.2f} kg")



Emission_kg_N2O: 0.00 kg
Emission_kg_NH3: 0.00 kg
Emission_kg_NO3: 0.01 kg
Emission_kg_NOx_N: 0.00 kg
Emission_kg_P: 0.00 kg


In [15]:
bd.projects.set_current('ecoinvent-3.10-cutoff')

fert_emission_names_compartment = {
    'name': ["Dinitrogen monoxide", "Ammonia", "Nitrate", "Nitrogen oxides", "Phosphate"],
    'categories': ["air", "air", "water", "air", "water"]
}

# Function to search and filter emissions in the database
def get_emission_info(fert_emission_names_compartment, database_name='ecoinvent-3.10-biosphere'):
    db = bd.Database(database_name)
    emission_info = []

    for name, category in zip(fert_emission_names_compartment['name'], fert_emission_names_compartment['categories']):
        # Search for the emission name in the database
        search_results = db.search(name)
        
        # Filter results to find the simplest category
        for result in search_results:
            result_dict = result.as_dict()
            if result_dict['categories'] == (category,):
                # Extract required fields
                emission_info.append({
                    'name': result_dict['name'],
                    'code': result_dict['code'],
                    'categories': result_dict['categories'],
                    'type': result_dict['type'],
                    'database': result_dict['database'],
                })
                break  # Stop searching once the simplest category is found

    return emission_info

emission_data = get_emission_info(fert_emission_names_compartment)
print(emission_data)

[{'name': 'Dinitrogen monoxide', 'code': '20185046-64bb-4c09-a8e7-e8a9e144ca98', 'categories': ('air',), 'type': 'emission', 'database': 'ecoinvent-3.10-biosphere'}, {'name': 'Ammonia', 'code': '87883a4e-1e3e-4c9d-90c0-f1bea36f8014', 'categories': ('air',), 'type': 'emission', 'database': 'ecoinvent-3.10-biosphere'}, {'name': 'Nitrate', 'code': '5189de76-6bbb-44ba-8c42-5714f1b4371f', 'categories': ('water',), 'type': 'emission', 'database': 'ecoinvent-3.10-biosphere'}, {'name': 'Nitrogen oxides', 'code': 'c1b91234-6f24-417b-8309-46111d09c457', 'categories': ('air',), 'type': 'emission', 'database': 'ecoinvent-3.10-biosphere'}, {'name': 'Phosphate', 'code': 'c8791f3c-3c4a-4278-91c0-483797d14da2', 'categories': ('water',), 'type': 'emission', 'database': 'ecoinvent-3.10-biosphere'}]


In [11]:
#This block is used to calculate emissions from pesticide application.

# This is example data of pesticides for the sake of checking if the code works. THIS MUST BE REPLACED WITH THE DATA FROM NICO.
pesticide_quantity_dict = {
    'pesticide_name': ['2,4-D', '2,4-DB', 'CARBARYL', 'CHLORPYRIFOS', 'CLETHODIM', 'CYHALOTHRIN-LAMBDA'],
    'pest_quantity_kg_per_ha': [1, 2, 3, 4, 5, 6]
}
df_pesticide_quantity = pd.DataFrame(pesticide_quantity_dict)

# Emission factors
EF_pest_soil = 0.9
EF_pest_air = 0.1

# Initialize an empty list to store the results
results = []

# Iterate through the DataFrame rows
for index, row in df_pesticide_quantity.iterrows():
    pesticide_name = row['pesticide_name']
    pest_quantity = row['pest_quantity_kg_per_ha']
    
    # Calculate emissions
    pest_em_soil = pest_quantity * EF_pest_soil
    pest_em_air = pest_quantity * EF_pest_air
    
    # Append the results as a dictionary
    results.append({
        'pesticide_name': pesticide_name,
        'pest_em_soil': pest_em_soil,
        'pest_em_air': pest_em_air
    })

# Convert the results list to a DataFrame
df_emissions = pd.DataFrame(results)

# Display the resulting DataFrame
print(df_emissions)


       pesticide_name  pest_em_soil  pest_em_air
0               2,4-D           0.9          0.1
1              2,4-DB           1.8          0.2
2            CARBARYL           2.7          0.3
3        CHLORPYRIFOS           3.6          0.4
4           CLETHODIM           4.5          0.5
5  CYHALOTHRIN-LAMBDA           5.4          0.6


In [ ]:
def create_main_process(name_database,Latitude,Longitude,crop):
    country = country_from_coordinates(Latitude,Longitude)
    name = "Production of "+crop+" in "+country
    data = {
    "code": uuid.uuid4().hex ,
    "name": name,
    "reference product":crop,
    "location": country,
    "Latitude": Latitude, 
    "Longitude": Longitude, 
    "database":name_database,
}
    self.product = db.new_node(**data)
    self.product.save()

In [17]:
# Example DataFrame structure for inputs and outputs
data = {
    'amount': [1, 2, 3, 4, 5, 6],
    'type': ['technosphere', 'technosphere', 'technosphere', 'biosphere', 'biosphere', 'biosphere'],
    'input_name': ['2,4-D', '2,4-DB', 'CARBARYL', 'CHLORPYRIFOS', 'CLETHODIM', 'CYHALOTHRIN-LAMBDA']
}
df_product_lci = pd.DataFrame(data)

# Define the create_edge function
def create_edge(amount, type_, input_name):
    # Hypothetical edge creation process
    edge = {
        'amount': amount,
        'type': type_,
        'input': input_name
    }
    print(f"Creating edge: {edge}")
    return edge

# Define the function to process the DataFrame and create edges
def process_io_dataframe(df_product_lci):
    edges = []
    for index, row in df_io.iterrows():
        amount = row['amount']
        type_ = row['type']
        input_name = row['input_name']
        
        edge = create_edge(amount, type_, input_name)
        edges.append(edge)
    
    return edges

# Process the DataFrame to create edges and return them
edges = process_io_dataframe(df_product_lci)

# Display the created edges
for edge in edges:
    print(edge)


Creating edge: {'amount': 1, 'type': 'technosphere', 'input': '2,4-D'}
Creating edge: {'amount': 2, 'type': 'technosphere', 'input': '2,4-DB'}
Creating edge: {'amount': 3, 'type': 'technosphere', 'input': 'CARBARYL'}
Creating edge: {'amount': 4, 'type': 'biosphere', 'input': 'CHLORPYRIFOS'}
Creating edge: {'amount': 5, 'type': 'biosphere', 'input': 'CLETHODIM'}
Creating edge: {'amount': 6, 'type': 'biosphere', 'input': 'CYHALOTHRIN-LAMBDA'}
{'amount': 1, 'type': 'technosphere', 'input': '2,4-D'}
{'amount': 2, 'type': 'technosphere', 'input': '2,4-DB'}
{'amount': 3, 'type': 'technosphere', 'input': 'CARBARYL'}
{'amount': 4, 'type': 'biosphere', 'input': 'CHLORPYRIFOS'}
{'amount': 5, 'type': 'biosphere', 'input': 'CLETHODIM'}
{'amount': 6, 'type': 'biosphere', 'input': 'CYHALOTHRIN-LAMBDA'}


In [ ]:
def create_edge(amount, name, code):
    self.product.new_edge(
        amount=amount, 
        type= type(name), #function that takes the name, returns technosphere or biosphere
        input=code #key, and the key is the database which has the code
    ).save()